<a href="https://colab.research.google.com/github/NikoDhrmwn/PadE/blob/main/PadE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
train_dir="/content/drive/MyDrive/RiceLeafs/train"
val_dir="/content/drive/MyDrive/RiceLeafs/validation"
Labels=[]

for file in os.listdir(train_dir):
    Labels+=[file]

Label=[]

for i in range(len(Labels)):
    Label+=[i]
    
mapping = dict(zip(Labels, Label)) 
reverse_mapping = dict(zip(Label, Labels)) 

def mapper(value):
    return reverse_mapping[value]

dataset=[]
valset=[]
count=0
for file in os.listdir(train_dir):
    path=os.path.join(train_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image=img_to_array(image)
        image=image/255.0
        dataset+=[[image,count]]
        t+=1
    count=count+1
count=0
for file in os.listdir(val_dir):
    path=os.path.join(val_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image=img_to_array(image)
        image=image/255.0
        valset+=[[image,count]]
        t+=1
    count=count+1

In [ ]:
data,labels0=zip(*dataset)
test,testlabels0=zip(*valset)
labels1=to_categorical(labels0)
labels=np.array(labels1)
data=np.array(data)
test=np.array(test)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(2498, 80, 80, 3)
(625, 80, 80, 3)
(2498, 4)
(625, 4)


In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=35,zoom_range=0.7,
                        width_shift_range=0.3,height_shift_range=0.1,shear_range=0.2,fill_mode="nearest")

In [ ]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

history = model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=15)

Epoch 1/15
79/79 [==============================] - 19s 230ms/step - loss: 1.3264 - accuracy: 0.4027 - val_loss: 1.2767 - val_accuracy: 0.4320
Epoch 2/15
79/79 [==============================] - 18s 225ms/step - loss: 1.2342 - accuracy: 0.4540 - val_loss: 1.2541 - val_accuracy: 0.4384
Epoch 3/15
79/79 [==============================] - 18s 221ms/step - loss: 1.2050 - accuracy: 0.4604 - val_loss: 1.0687 - val_accuracy: 0.5712
Epoch 4/15
79/79 [==============================] - 17s 220ms/step - loss: 1.1572 - accuracy: 0.5076 - val_loss: 1.1299 - val_accuracy: 0.5248
Epoch 5/15
79/79 [==============================] - 18s 222ms/step - loss: 1.1311 - accuracy: 0.5208 - val_loss: 1.0567 - val_accuracy: 0.5776
Epoch 6/15
79/79 [==============================] - 18s 228ms/step - loss: 1.1323 - accuracy: 0.5296 - val_loss: 1.0523 - val_accuracy: 0.5616
Epoch 7/15
79/79 [==============================] - 18s 227ms/step - loss: 1.0974 - accuracy: 0.5472 - val_loss: 1.0482 - val_accuracy: 0.5648

In [ ]:
 model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
image=load_img("/content/drive/MyDrive/RiceLeafs/validation/BrownSpot/IMG_20190419_113242.jpg",target_size=(80,80))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

Prediction is LeafBlast.
